In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from kaggle_environments import make, evaluate
import numpy as np

envi = make('connectx', debug = True)

In [ ]:
def agent_vinod(obs, config):
    def get_next_board(grid, col, piece):
        next_gnid = grid.copy()
        for row in range(next_gnid.shape[0]-1, -1, -1):
            if next_gnid[row][col] == 0:
                break
        next_gnid[row][col] = piece
        return next_gnid

    def check_window(window, piece, num_disc, config):
        return (window.count(piece) == num_disc and window.count(0) == config.inarow-num_disc)


    def count_windows(grid, config, piece, num_disc):
        num_windows = 0
    #     #Horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row][col:col+config.inarow])
                if check_window(window, piece, num_disc, config):
                    num_windows+=1

        #Vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window(window, piece, num_disc, config):
                    num_windows+=1
        #+ve diagonal
        for row in range(config.rows-config.inarow+1):
            for col in range(config.columns-config.inarow+1):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window(window, piece, num_disc,config):
                    num_windows += 1

        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window(window, piece, num_disc,config):
                    num_windows += 1

        return num_windows


    def get_heuristic(grid, mark, config):
        num_twos = count_windows(grid, config, mark, 2)
        num_threes = count_windows(grid, config, mark,3)
        num_fours = count_windows(grid, config, mark, 4)
        num_twos_opp = count_windows(grid,  config, mark%2+1, 2)
        num_threes_opp = count_windows(grid, config, mark%2+1, 3)
        score = 1e10*num_fours + 1e4*num_threes + 1e2*num_twos -1*num_twos_opp -1e6*num_threes_opp
        return score
        
#         num_threes = count_windows(grid, config, mark, 3)
#         num_fours = count_windows(grid, config, mark, 4)
#         num_threes_opp = count_windows(grid, config, mark%2+1, 3)
#         score = num_threes - 1e2*num_threes_opp + 1e6*num_fours
#         return score

    def scoree(grid, mark, config, col):
        next_grid = get_next_board(grid, col, mark)
        score = get_heuristic(next_grid, mark, config)
        return score
    
    import random
    import numpy as np
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    score = dict(zip(valid_moves, [scoree(grid, obs.mark, config, col) for col in valid_moves]))
    max_score = [key for key in score.keys() if score[key] == max(score.values())]
    return random.choice(max_score)

In [ ]:
envi.play([agent_vinod, None], width = 500, height = 500)